# Laden der Daten
In diesem Notebook werden die Daten eingelesen, welche direkt über die URL zugängig sind. Die Geodaten für das Weg- und Velonetz <br>
in Zürich müssen manuell auf der Open Data Seite runtergeladen und dann über einen CSV-Import geladen werden, nachdem die Datei Entzipped wurde.<br>
Grund hierfür ist die Auswahl des Ausschnitts, in welchem das Velonetz angezeigt werden soll. <br>
Dieser muss auf der Seite ausgewählt werden und es wird dann ein Downlaodlink für 5 Minuten zur Verfügung gestellt.


## Verwendete Links/Quellen

In [1]:
urls = []
urls.append("https://www.hev-schweiz.ch/vermieten/nebenkostenabrechnungen/sonnenscheindauer")
urls.append("https://www.hev-schweiz.ch/vermieten/statistiken/mietpreise/durchschnittliche-mietpreise")

url3 = "https://opendata.swiss/de/dataset/median-strompreis-per-kanton"

# Installation notwendiger Bibliotheken

In [2]:
!pip install --upgrade pip
!pip install beautifulsoup4
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


# Module importieren

In [3]:
from bs4 import BeautifulSoup as Soup
import urllib.request
import openpyxl
import pandas as pd
from pathlib import Path
from google.colab import drive # Library to access shared folder "ie_scripting_datasets" on google drive
import os

# Definition Colab Speicherort und Access

In [4]:
drive.mount('/content/drive') #will give access to "ie_scripting_datasets" on google drive in folder selection

Mounted at /content/drive


# Funktionen

Funktion für das extrahieren von Tabellendaten

In [5]:
def tableDataText(table):
    def rowgetDataText(tr, coltag='td'):
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow:
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs:
        rows.append(rowgetDataText(tr, 'td') )
    return rows

Herunterladen der Tabellen aus dem Web und umwanldung in ein Pandas DataFrame

In [6]:
html = urllib.request.urlopen(urls[0]).read()
soup = Soup(html, parser='lxml')
htmltables = soup.find_all('table', { 'class' : 'contenttable acc_content ce-table' })

all_dataframes = []
for table in htmltables:
  table = tableDataText(table)
  df = pd.DataFrame(table)
  all_dataframes.append(df)

Erstellen der Excel-Datei und speichern der Tabellen in einzelne Register

In [7]:
filepath = Path('/content/drive/My Drive/ie_scripting_datasets/Sonnenscheindauer.xlsx')
filepath.parent.mkdir(parents=True, exist_ok=True)

year = 2024
with pd.ExcelWriter(filepath) as writer:
  for dataframe in all_dataframes:
    dataframe.to_excel(writer, sheet_name=str(year), index=False, header=False)
    year = year - 1